# MAPAS

In [2]:
import pandas as pd
import folium
from folium.plugins import HeatMap # Mapa de calor
from folium import CustomIcon
#pip install geopandas

archivo_filename = "C:\\xampp\\htdocs\\datalpine\\resources\\db\\Scrapining\\extra_db\\geocodificado_agosto_tulancingo.csv"
df = pd.read_csv(archivo_filename)

# Distribución de Propiedades 

In [101]:
# Calcular la ubicación promedio de todas las propiedades (centroide)
lat_centro = df['Latitud'].mean()
lon_centro = df['Longitud'].mean()

# Crear el mapa centralizado en el centroide
mapa = folium.Map(location=[lat_centro, lon_centro], zoom_start=10)

# Establecer los límites de desplazamiento (por ejemplo, un rango de ±0.5 grados en latitud y longitud)
mapa.fit_bounds([[lat_centro - 0.5, lon_centro - 0.5], [lat_centro + 0.5, lon_centro + 0.5]])

coordenadas = df[['Latitud', 'Longitud']].values.tolist()
HeatMap(coordenadas, radius=15, blur=10, min_opacity=0.2).add_to(mapa)

# Configurar los límites de desplazamiento para evitar que el usuario salga del área
mapa.options['maxBounds'] = [[lat_centro - 0.5, lon_centro - 0.5], [lat_centro + 0.5, lon_centro + 0.5]]
mapa.options['minZoom'] = 11  # Limita el zoom out al nivel de zoom 10

# Guardar el mapa en HTML
mapa.save("assets/mapas/propiedades.html")
mapa

# Distribución de Colonias

### COLONIAS ESTANDARIZADAS:

In [59]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

In [106]:
# Crear el mapa centrado en la ubicación general de las colonias
mapa = folium.Map(location=[df['Latitud'].mean(), df['Longitud'].mean()], zoom_start=12)

# Agrupar por colonia y calcular métricas adicionales
colonias_data = df.groupby('Colonia_Estandarizada').agg(
    #num_propiedades=('propiedad', 'size'),
    precio_promedio=('precio', 'mean'),
    precio_mediano=('precio', 'median'),
    precio_m2=('precio_m2_terreno', 'mean'),
    baños=('bano_total', 'mean'),
    recamaras=('recamaras', 'mean'),
    estacionamientos=('estacionamientos', 'mean'),
    area_construida=('metros_construido', 'mean'),
    area_total=('metros_total', 'mean'),
    latitud_media=('Latitud', 'mean'),
    longitud_media=('Longitud', 'mean')
).reset_index()

# Categorizar el precio promedio usando el diccionario de rangos
def categorizar_precio(precio_promedio):
    for categoria, (min_precio, max_precio) in rangos_precio.items():
        if min_precio <= precio_promedio <= max_precio:
            return categoria
    return "Sin Categoría"  # En caso de que no encaje en ningún rango
colonias_data['categoria_precio'] = colonias_data['precio_promedio'].apply(categorizar_precio)


# Crear el mapa centrado en la ubicación general de las colonias
mapa = folium.Map(location=[df['Latitud'].mean(), df['Longitud'].mean()], zoom_start=12)

# Ruta del archivo PNG para el icono
icon_path = "../../../public/Admin/assets/images/clip_casa.png"

# Añadir marcadores para cada colonia con icono personalizado
for idx, row in colonias_data.iterrows():
    colonia_titulo = row['Colonia_Estandarizada'].capitalize()
    popup_content = f"""
    <div style="
        width: 200px; 
        font-family: Arial, sans-serif; 
        padding: 10px; 
        border-radius: 8px; 
        background-color: #f9f9f9;
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        color: #333;
    ">

    <h4 style="margin-bottom: 8px; color: #2931f5; font-family: 'Arial', sans-serif; font-weight: bold;">{colonia_titulo}</h4>
    <div >
            
        </div>
        <div style="margin-bottom: 8px;">
            <strong>Precio: <span style="color: rgb(205, 213, 0);">${row['precio_promedio']:.2f} MXN</span></strong> <br>
            <strong>Categoría: <span style="color: #bfc1e2;">{row['categoria_precio']}</span></strong>
            <br><strong  style="border-top: 1px solid #ddd; margin: 8px 0;font-size: 0.9em; color: #555;">Precio por m²:</strong> ${row['precio_m2']:.2f} MXN
        </div>
        <hr style="border-top: 1px solid #ddd; margin: 8px 0;">
        <div style="font-size: 0.9em;">
            <strong>Baños:</strong> {int(row['baños'])}<br>
            <strong>Recámaras:</strong> {int(row['recamaras'])}<br>
            <strong>Estacionamientos:</strong> {int(row['estacionamientos'])}
        </div>
        <hr style="border-top: 1px solid #ddd; margin: 8px 0;">
        <div style="font-size: 0.9em;">
            <strong>Área de terreno:</strong> {row['area_total']:.2f} m²<br>
            <strong>Área construida:</strong> {row['area_construida']:.2f} m²
        </div>
    </div>
    """
    popup = folium.Popup(popup_content, max_width=250)
    
    # Configurar el icono personalizado
    custom_icon = CustomIcon(
        icon_image=icon_path,
        icon_size=(32, 32),   # Ajusta el tamaño del icono según sea necesario
        icon_anchor=(16, 16),  # Centra el icono, ajusta según el tamaño del icono
    )

    # Añadir el marcador con el icono personalizado
    folium.Marker(
        location=[row['latitud_media'], row['longitud_media']],
        popup=popup,
        tooltip=colonia_titulo,
        icon=custom_icon
    ).add_to(mapa)

# Guardar el mapa en HTML (si lo deseas)
mapa.save("assets/mapas/colonias.html")
mapa

# Clientes potenciales foraneos

In [2]:
# Asignar puntajes basados en los datos proporcionados
import pandas as pd
data = {
    "Ciudad": ["USA", "CDMX","Venezuela","Surinam", "México", "Hidalgo", "Puebla", "Nuevo león", "Nigueria"],
    "PIB per cápita": [4,5,5,5,2,2,2,5,5],
    "Demografía": [5,2,5,2,5,2,5,3,5],
    "Grupos de edad": [5,5,5,5,4,3,2,4,4],
    "Tasa de Crecimiento Poblacional":[1,1,3,3,3,3,3,5,5],
    "Distancia a lugar": [1,4,1,1,3,5,4,2,1],
    "Interés (%)": [2,3,4,3,2,5,2,1,2],
}

# Ponderaciones por variable (ajustadas según importancia)
weights = {
    "PIB per cápita": 0.25,
    "Demografía": 0.10,
    "Grupos de edad": 0.15,
    "Tasa de Crecimiento Poblacional": 0.05,
    "Distancia a lugar": 0.05,
    "Interés (%)": 0.40,
}

# Calcular puntaje total ponderado
df = pd.DataFrame(data)
df["Puntuación Total"] = (
    df["PIB per cápita"] * weights["PIB per cápita"] +
    df["Demografía"] * weights["Demografía"] +
    df["Grupos de edad"] * weights["Grupos de edad"] +
    df["Tasa de Crecimiento Poblacional"] * weights["Tasa de Crecimiento Poblacional"] +
    df["Distancia a lugar"] * weights["Distancia a lugar"] +
    df["Interés (%)"] * weights["Interés (%)"]
)

print(df[["Ciudad", "Puntuación Total"]])

       Ciudad  Puntuación Total
0         USA              3.15
1        CDMX              3.65
2   Venezuela              4.30
3     Surinam              3.60
4      México              2.70
5     Hidalgo              3.55
6      Puebla              2.45
7  Nuevo león              2.90
8    Nigueria              3.45


#### Adignación de geocodificación a los lugares

In [3]:
import pandas as pd
import time
import urllib3
import json
# Función de geocodificación usando la API de ArcGIS
def geocode_arcgis(city_name):
    url = "https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/findAddressCandidates"
    params = {
        "f": "json",
        "outFields": "*",
        "maxLocations": 1,
        "singleLine": city_name,
        "outSr": "4326",
        "forStorage": "false"
    }
    try:
        http = urllib3.PoolManager()
        response = http.request('GET', url, fields=params)
        data = response.data.decode('utf-8')
        if data:
            data = json.loads(data)
            if data['candidates']:
                location = data['candidates'][0]['location']
                return location['y'], location['x']
    except urllib3.exceptions.RequestError as e:
        print(f"Error al geocodificar {city_name}: {e}")
    return None, None  # En caso de error o sin resultados válidos

# Aplicar la geocodificación a cada ciudad
latitudes = []
longitudes = []

for idx, row in df.iterrows():
    city_name = row['Ciudad']
    lat, lon = geocode_arcgis(city_name)
    latitudes.append(lat)
    longitudes.append(lon)
    time.sleep(0.5)  # Espera para evitar sobrecarga en la API

# Añadir columnas de latitud y longitud al DataFrame
df['Latitud'] = latitudes
df['Longitud'] = longitudes

# Almacenar el resultado en un DataFrame
geocoded_df = df[['Ciudad', 'Latitud', 'Longitud','Puntuación Total']]
print(geocoded_df)

       Ciudad    Latitud    Longitud  Puntuación Total
0         USA  39.398703  -99.414619              3.15
1        CDMX  19.432677  -99.134211              3.65
2   Venezuela   7.120734  -66.162820              4.30
3     Surinam   3.931777  -56.013608              3.60
4      México  23.952825 -102.548353              2.70
5     Hidalgo  20.479558  -98.887063              3.55
6      Puebla  19.037930  -98.203460              2.45
7  Nuevo león  25.572595  -99.968972              2.90
8    Nigueria  12.038605    8.258817              3.45


In [5]:
import folium
from folium.plugins import HeatMap # Mapa de calor
from folium import CustomIcon

# Calcular el centroide de todas las ciudades
latitudes = geocoded_df['Latitud'].tolist()
longitudes = geocoded_df['Longitud'].tolist()
centroide_lat = sum(latitudes) / len(latitudes)
centroide_lon = sum(longitudes) / len(longitudes)

# Crear el mapa centrado en el centroide con tiles en español
m = folium.Map(location=[centroide_lat, centroide_lon], zoom_start=6, tiles="cartodbpositron", language="es")

# Agregar marcadores para cada ciudad según su puntuación total
for _, row in geocoded_df.iterrows():
    folium.CircleMarker(
        location=[row['Latitud'], row['Longitud']],
        radius=row['Puntuación Total'] *7,  # Ajusta el tamaño
        color='#cdd500' if row['Puntuación Total'] > 3 else '#2931f5',
        fill=True,
        fill_opacity=0.6,
        popup=f"{row['Ciudad']}",
        #{row['Puntuación Total']}
    ).add_to(m)

# Guardar el mapa en HTML (si lo deseas)
m.save("C:\\xampp\\htdocs\\datalpine\\resources\\Ciudades\\Tulancingo\\assets\\mapas\\foraneos.html")
m